In [19]:
import pandas as pd
from collections import Counter, defaultdict

In [33]:
# Iterate over a list in chunks of size n. Return tuples (for dict).
def chunks(iterable, n):
    for ix, item in enumerate(iterable):
        if ix == len(iterable) - (n-1): return
        yield tuple(iterable[ix:ix+n])
        
# Build the conditional probability tables.
def condProbTables(ngramfreqs, nngramfreqs):
    nprobs = defaultdict(int)
    prevnngramnexts = defaultdict(list)
    for ngram, freq in ngramfreqs.items():
        prevnngram = ngram[:-1]
        currchar = ngram[-1]
        nprobs[(currchar, prevnngram)] = float(ngramfreqs[ngram]) / nngramfreqs[prevnngram]
        if prevnngram not in prevnngramnexts.keys():
            prevnngramnexts[prevnngram].extend([(currchar, (float(ngramfreqs[ngram]) / nngramfreqs[prevnngram]))])
            continue
        prevnngramnexts[prevnngram].extend([(currchar, (float(ngramfreqs[ngram]) / nngramfreqs[prevnngram]))])
    return nprobs, prevnngramnexts

In [25]:
oscar = pd.read_csv('oscar2notes.txt', skiprows=2)[:].sort_values("Offset")
oscar.index = range(1, len(oscar) + 1)
oscar = oscar[oscar.Octave >= 4]
oscar.head()

,Note/Rest,Octave,Len,Offset
2,D,5,0.750000,12.666667
3,E,4,0.666667,14.000000
4,C#,5,0.875000,14.000000
5,A,5,0.250000,15.000000
6,F,4,3.125000,16.000000


In [29]:
possiblenotes = ["%s%s" % (row[1]["Note/Rest"], row[1]["Octave"]) for row in oscar.iterrows()]
possiblenotes.insert(0, "start")
possiblenotes.insert(0, "start")
possiblenotes.insert(0, "start")

In [32]:
bigramfreqs = defaultdict(int)
for i in chunks(possiblenotes, 2):
    bigramfreqs[i] += 1
trigramfreqs = defaultdict(int)
for i in chunks(possiblenotes, 3):
    trigramfreqs[i] += 1
quadgramfreqs = defaultdict(int)
for i in chunks(possiblenotes, 4):
    quadgramfreqs[i] += 1

In [34]:
# Encode ngram probabilities
triprobs, prevbigramnexts = condProbTables(trigramfreqs, bigramfreqs)
# quadprobs, prevtrigramnexts = condProbTables(quadgramfreqs, trigramfreqs)